In [25]:
import sys
sys.path.append('C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive')
print(sys.path)  # Esto mostrará todas las rutas en sys.path

from model.preprocessing.data_cleaning import DataCleaner
from model.utils.data_analysis import DataAnalyzer
#from model.preprocessing.data_extraction import DataExtractor
from model.preprocessing.feature_extraction import FeatureSelector,FeatureVisualizer



['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv', '', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\Pythonwin', 'C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive', 'C:/Users/juans/OneDrive/Documento

In [82]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import unidecode
import csv

class DataExtractor:
    def __init__(self, base_path="../../data/", columns_to_drop=None):
        self.base_path = base_path
        self.columns_to_drop = columns_to_drop if columns_to_drop is not None else []
        self.removed_features = []  # Almacena los nombres de las características eliminadas
        self.removed_rows = []  # Almacena las filas problemáticas

    def preprocess_csv(self, input_path, output_path, encoding='latin1'):
        with open(input_path, 'r', encoding=encoding) as infile, \
            open(output_path, 'w', encoding=encoding) as outfile:
            current_line = []
            for line in infile:
                # Comprobar si la línea comienza con un número de cinco o más dígitos
                if line.strip() and line[:5].isdigit() and current_line:
                    # Escribir la línea acumulada en el archivo de salida
                    outfile.write(''.join(current_line).replace('\n', ' ') + '\n')
                    current_line = [line]  # Comenzar una nueva línea acumulada
                else:
                    # Continuar acumulando segmentos de la línea actual
                    current_line.append(line)
            # Escribir la última línea acumulada
            if current_line:
                outfile.write(''.join(current_line).replace('\n', ' ') + '\n')

    def load_data(self, nameFolderRaw, file_name, processed_file_name='processed.csv', encoding='latin1', delimiter=';'):
        original_path = self.base_path + nameFolderRaw + file_name
        processed_path = self.base_path + nameFolderRaw + processed_file_name
        # Preprocesar el archivo para limpiar los saltos de línea incorrectos
        self.preprocess_csv(original_path, processed_path, encoding=encoding)
        
        try:
            data = pd.read_csv(processed_path, 
                               encoding=encoding, 
                               delimiter=delimiter, 
                               header=0,
                               quoting=csv.QUOTE_NONE,
                               on_bad_lines='skip')
        except pd.errors.ParserError as e:
            print(f"Error al leer el archivo: {e}")
            return None

        object_columns = data.select_dtypes(include=[object]).columns
        for col in object_columns:
            data[col] = data[col].apply(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

        if self.columns_to_drop:
            data.drop(columns=self.columns_to_drop, inplace=True, errors='ignore')

        return data


    def get_removed_features(self):
        return self.removed_features

    def get_removed_rows(self):
        return self.removed_rows

    @staticmethod
    def save_data_pickle(data, namefile, base_path, filter_condition=None):
        if filter_condition:
            data = data.query(filter_condition)
        with open(base_path + "processed/" + namefile, 'wb') as f:
            pickle.dump(data, f)

    @staticmethod
    def extract_test_validation_training_data(data, target_column='Deterioro Cognitivo', training_ratio=0.7, test_ratio=0.2, validation_ratio=0.1):
        if target_column not in data.columns:
            raise ValueError(f"La columna objetivo '{target_column}' no se encuentra en los datos")

        training_data, temp_data = train_test_split(data, test_size=1-training_ratio, random_state=42)
        validation_data, test_data = train_test_split(temp_data, test_size=(validation_ratio / (test_ratio + validation_ratio)), random_state=42)

        return {
            'train': (training_data.drop(columns=[target_column]), training_data[target_column]),
            'validation': (validation_data.drop(columns=[target_column]), validation_data[target_column]),
            'test': (test_data.drop(columns=[target_column]), test_data[target_column])
        }


In [83]:
# Ejemplo de uso
datahandler = DataExtractor()
data = datahandler.load_data("raw/", "BD sin RM(Base de datos) (2).csv")
if data is not None:
    print(data.head())
else:
    print("No se pudo cargar los datos.")

C:\Users\juans\AppData\Local\Temp\ipykernel_39896\1936011678.py:38: DtypeWarning: Columns (44,67,95,102,174,175,211,213,231,232,234,257,258,260,265,267) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(processed_path,


  Identificación  Lugar Nacimiento  Municipio Atención  Edad  Género  \
0       10035213                 0                 1.0  63.0     0.0   
1       10055205                 2                 3.0  79.0     0.0   
2       10056901                 3                 3.0  73.0     0.0   
3       10061985                 6                 3.0  71.0     0.0   
4       10065630                 6                 3.0  69.0     0.0   

   Grupo Sanguíneo  Lateralidad  Autopercepción Salud  Estado Civil  \
0              7.0          1.0                     1           1.0   
1              6.0          1.0                     1           0.0   
2              7.0          1.0                     1           1.0   
3              0.0          1.0                     1           2.0   
4              0.0          1.0                     0           1.0   

  Vive con pareja  ...  Cdr Orientacion Cdr Juicio  Cdr Vida Social  \
0               1  ...              0.0        0.5                0  

In [86]:
print(data.info(max_cols=3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Columns: 268 entries, Identificación to Médico Atención 
dtypes: float64(198), int64(4), object(66)
memory usage: 5.5+ MB
None


In [81]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997 entries, 0 to 2996
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:


import pandas as pd
import numpy as np



# Instanciar y usar el preprocesador
preprocessor = DataCleaner(X, y)
data_analyzer = DataAnalyzer()


preprocessor.encode_features(target_column='income', method='onehot')

# Limpieza de duplicados y datos nulos
preprocessor.delete_null_data()
#preprocessor.delete_duplicate_rows()
#preprocessor.delete_Object_columns()

# Imputación de valores faltantes
#X_imputed = preprocessor.impute_missing_values()

# Estandarización
preprocessor.standardize()

# normalización
preprocessor.normalize_features()

#detectar y eliminar outliers
preprocessor.delete_outliers().value_counts()

#balancear datos
preprocessor.balance_data()

#generar columnas polinomiales
#xfold = preprocessor.generate_polynomial_columns()

#preprocessor.reduce_dimensionality(n_components=2,replace=True)

data = pd.concat([preprocessor.X, preprocessor.y], axis=1)
print(data_analyzer.describe_data(data=data))
data_analyzer.visualize_correlation(data=data)
data_analyzer.visualize_data(data, kind='boxplot')
data_analyzer.visualize_data(data, kind='histogram')




In [ ]:
X = preprocessor.X
y = preprocessor.y


# Inicializar el selector de características
selector = FeatureSelector(X, y)

# 1. Selección de características con Chi-squared
chi2_result = selector.select_features_chi2(k=14)
FeatureVisualizer.plot_scores(chi2_result['features'], chi2_result['scores'], title='Chi-squared Scores')

# 2. Selección de características con Spearman
spearman_result = selector.select_features_spearman(threshold=0.01)
FeatureVisualizer.plot_correlations(spearman_result['selected_features'], spearman_result['correlations'], title='Spearman Correlation Coefficients')

# 3. Selección de características con Lasso
lasso_result = selector.lasso_feature_selection(alpha=0.01)
FeatureVisualizer.plot_coefficients(lasso_result['selected_features'], lasso_result['coefficients'], title='Lasso Coefficients')

# 4. Selección de características con Sequential Feature Selector
sfs_result = selector.sequential_feature_selection(n_features_to_select=13, direction='forward')
print("Sequential Feature Selection Selected Features:", sfs_result['selected_features'])

# 5. Importancia de características con Random Forest
rf_result = selector.random_forest_feature_importance(n_features=14)
FeatureVisualizer.plot_importances(rf_result['selected_features'], rf_result['importances'], title='Random Forest Feature Importances')